In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
import datetime

model = YOLO('yolov10n.pt')
cap = cv2.VideoCapture("testing.mp4")
data = pd.read_csv("traffic.csv")
CAR_LABEL = "car"
BIKE_LABEL = "motorcycle"
TRUCK_LABEL = "truck"

def visualize_and_count(image, results):
    car_count = 0
    bike_count = 0
    truck_count = 0

    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            label = model.names[cls]
            conf = box.conf[0]

            if label == CAR_LABEL:
                car_count += 1
            elif label == BIKE_LABEL:
                bike_count += 1
            elif label == TRUCK_LABEL:
                truck_count += 1

            bbox = box.xyxy[0].cpu().numpy().astype(int)
            start_point = (bbox[0], bbox[1])
            end_point = (bbox[2], bbox[3])
            cv2.rectangle(image, start_point, end_point, (0, 255, 0), 2)
            cv2.putText(image, label, start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.putText(image, f"Cars : {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(image, f"Bikes : {bike_count}", (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(image, f"Trucks : {truck_count}", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return image, car_count, bike_count, truck_count

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    annotated_frame, car_count, bike_count, truck_count = visualize_and_count(frame, results)
    total = car_count + bike_count + truck_count
    print(f"Number of cars in this frame: {car_count}")
    cv2.imshow('Object Detection', annotated_frame)

    data.loc[len(data)]  =  [car_count, bike_count, truck_count , total , datetime.datetime.now()]
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

data.to_csv("traffic.csv", index=False)

cap.release()
cv2.destroyAllWindows()


0: 384x640 7 cars, 1 truck, 1 umbrella, 123.4ms
Speed: 4.0ms preprocess, 123.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Number of cars in this frame: 7

0: 384x640 7 cars, 1 truck, 1 umbrella, 101.4ms
Speed: 2.4ms preprocess, 101.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Number of cars in this frame: 7

0: 384x640 7 cars, 1 umbrella, 100.9ms
Speed: 1.0ms preprocess, 100.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Number of cars in this frame: 7

0: 384x640 7 cars, 1 truck, 1 umbrella, 101.9ms
Speed: 2.5ms preprocess, 101.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Number of cars in this frame: 7

0: 384x640 7 cars, 1 bus, 1 umbrella, 93.9ms
Speed: 2.0ms preprocess, 93.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Number of cars in this frame: 7

0: 384x640 8 cars, 1 umbrella, 97.3ms
Speed: 2.4ms preprocess, 97.3ms inference, 0.0ms postprocess per image at shape

In [1]:
import cv2
from ultralytics import YOLO
import pandas as pd
import datetime

model = YOLO('yolov10n.pt')

videos = {
    1: "T2.mp4",
    2: "testing.mp4",
    3: "testinglow.mp4"
}

data = pd.read_csv("traffic.csv")

CAR_LABEL = "car"
BIKE_LABEL = "motorcycle"
TRUCK_LABEL = "truck"

def visualize_and_count(image, results):
    car_count = 0
    bike_count = 0
    truck_count = 0

    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            label = model.names[cls]

            if label == CAR_LABEL:
                car_count += 1
            elif label == BIKE_LABEL:
                bike_count += 1
            elif label == TRUCK_LABEL:
                truck_count += 1

            bbox = box.xyxy[0].cpu().numpy().astype(int)
            start_point = (bbox[0], bbox[1])
            end_point = (bbox[2], bbox[3])
            cv2.rectangle(image, start_point, end_point, (0, 255, 0), 2)
            cv2.putText(image, label, start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.putText(image, f"Cars : {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(image, f"Bikes : {bike_count}", (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    cv2.putText(image, f"Trucks : {truck_count}", (50, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return car_count + bike_count + truck_count


for junction_id, video_file in videos.items():
    cap = cv2.VideoCapture(video_file)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        total_vehicles = visualize_and_count(frame, results)

        timestamp = datetime.datetime.now().strftime("%d-%m-%Y %H:%M")
        entry_id = f"{timestamp[:10].replace('-', '')}{junction_id}{str(len(data)+1).zfill(3)}"

        data.loc[len(data)] = [timestamp, junction_id, total_vehicles, entry_id]

        cv2.imshow(f'Junction {junction_id} - Object Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()

data.to_csv("traffic.csv", index=False)
cv2.destroyAllWindows()




Ultralytics YOLOv8.2.82  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Ultralytics YOLOv8.2.82  Python-3.9.19 torch-2.4.0+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)

YOLOv10n summary (fused): 285 layers, 2,762,608 parameters, 0 gradients, 8.6 GFLOPs
YOLOv10n summary (fused): 285 layers, 2,762,608 parameters, 0 gradients, 8.6 GFLOPs
0: 384x640 6 cars, 3 trucks, 152.9ms
Speed: 6.0ms preprocess, 152.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 1 umbrella, 156.7ms
Speed: 4.0ms preprocess, 156.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 134.8ms
Speed: 6.1ms preprocess, 134.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 3 trucks, 119.5ms
Speed: 4.0ms preprocess, 119.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 truck, 1 umbrella, 127.7ms
Speed: 3.3ms preprocess, 127.7ms inf

: 